In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

In [2]:
df = pd.read_csv("/workspaces/Data-Analyst-Project-Portfolio/Netflix-WatchPattern-Analytics/data/netflix_cleaned.csv")

In [3]:
df['Date'] = pd.to_datetime(df['Watch_Date'], errors='coerce')
df.dropna(subset=['Date'], inplace=True)

In [8]:
user_df = df.groupby('User_ID').agg({
    'Video_Title': 'count',                          
    'Date': [lambda x: x.nunique(), 'min', 'max'] 
})
user_df.columns = ['Total_Watches', 'Active_Days', 'First_Watch', 'Last_Watch']
user_df['Watch_Per_Day'] = user_df['Total_Watches'] / user_df['Active_Days']
user_df['Watch_Duration'] = (user_df['Last_Watch'] - user_df['First_Watch']).dt.days + 1
user_df['Watch_Frequency'] = user_df['Active_Days'] / user_df['Watch_Duration']

# Clean
user_df.replace([float("inf"), float("nan")], 0, inplace=True)
user_df.fillna(0, inplace=True)


In [9]:
features = ['Total_Watches', 'Watch_Per_Day', 'Watch_Frequency']
X = user_df[features]

# Scale data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)